### 0. Подготовка данных

In [2]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
import numpy as np
import pandas as pd

**Посмотрим на один из файлов с данными о координатах форм одного слайда хорошей презентации**

In [3]:
# Путь к данным
PATH_TO_DATA = 'data/nice_slide'

In [4]:
slide01_data = pd.read_csv(os.path.join(PATH_TO_DATA, 'slide01.csv'))

In [5]:
slide01_data.head()

,slide,shape,left,top,width,height,level
0,slide1,Picture 7,0,0,960,540,1
1,slide1,Rectangle 3,0,0,907,540,2
2,slide1,TextBox 2,87,102,543,95,3
3,slide1,TextBox 8,87,220,402,97,4
4,slide1,Picture 5,587,50,275,490,5


- **left** - расстояние в пунктах от левого края ограничивающей рамки фигуры до левого края слайда;
- **top** - расстояние в пунктах от верхнего края ограничивающей рамки фигуры до верхнего края слайда;
- **width** - ширина объекта в пунктах;
- **height** - высота объекта в пунктах.

1. Считаем данные из каждого файла и создадим один DataFrame
2. Для каждой формы (Shape) найдём
    - её уровень на слайде
    - координаты центра по ширине - **horiz_cent**
    - координаты центра по высоте - **vert_cent**
3. Определить видимость каждой формы: полностью ли она видна. Выставить маркер **visib=1**, если видна полностью, **visib = 0**, если закрыта сверху другими формами.

In [30]:
path_template = os.path.join(PATH_TO_DATA, os.path.normcase('slide*.csv'))
# создадим и отсортируем список файлов лексикографически
csv_files = glob(path_template)
csv_files.sort()
# создадим DataFrames из всех данных файлов csv_files
df = pd.DataFrame()
for file in csv_files:
    data = pd.read_csv(file)
    df = pd.concat([df, data])
# Найдем количество объектов на каждом слайде
data = (df.groupby('slide')['slide'].count()).to_frame(name='count')
df = pd.merge(left=df, right=data, on='slide')
# Инвертирeуем уровень: первый сверху слой будет иметь уровень 0
df['level'] = df['count'] - df['level']
df['horiz_cent'] = df.left + df.width // 2
df['vert_cent'] = df.top + df.height // 2
# Определим, виден ли объект полностью на слайде

### 1. Проверка гипотез

Проверим следующие гипотезы

1. Горизонтальный и вертикальный центр объектов, лежащих на самых верхних слоях близок к 1/3 и к 2/3 ширины и высоты презентации

In [31]:
df

,slide,shape,left,top,width,height,level,count,horiz_cent,vert_cent
0,slide1,Picture 7,0,0,960,540,8,9,480,270
1,slide1,Rectangle 3,0,0,907,540,7,9,453,270
2,slide1,TextBox 2,87,102,543,95,6,9,358,149
3,slide1,TextBox 8,87,220,402,97,5,9,288,268
4,slide1,Picture 5,587,50,275,490,4,9,724,295
...,...,...,...,...,...,...,...,...,...,...
136,slide11,Rectangle 14,98,78,426,207,4,11,311,181
137,slide11,TextBox 9,123,152,381,133,3,11,313,218
138,slide11,TextBox 10,122,100,336,51,2,11,290,125
139,slide11,Picture 13,82,318,43,43,1,11,103,339


In [ ]:
df

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 140
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   slide       141 non-null    object
 1   shape       141 non-null    object
 2   left        141 non-null    int64 
 3   top         141 non-null    int64 
 4   width       141 non-null    int64 
 5   height      141 non-null    int64 
 6   level       141 non-null    int64 
 7   count       141 non-null    int64 
 8   horiz_cent  141 non-null    int64 
 9   vert_cent   141 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 12.1+ KB


In [ ]:
def prepare_train_set(path_to_csv_files):
    path_template = os.path.join(path_to_csv_files, os.path.normcase('slide*.csv'))
    # создадим и отсортируем список файлов лексикографически
    csv_files = glob(path_template)
    csv_files.sort()
    # создадим список DataFrames из csv_files
    series = []
    for file in tqdm_notebook(csv_files):
        data = pd.read_csv(file)
        series.append(data['slide'])
    # посчитаем частоту - 'frequency'
    frequency_dataframe = series[0].append(series[1:]).value_counts()
    frequency = {key: (index, frequency_dataframe[key]) for index, key
                 in enumerate(frequency_dataframe.index, start=1)}
    # создаём список user_ids
    user_ids = [int(re.search('([1-9][0-9]*).csv$', path).group(1)) for path in csv_files]
    # словарь уникальных сайтов
    columns = ['site' + str(i) for i in range(1, session_length + 1)]
    features_list = []
    for index, sites in tqdm_notebook(enumerate(series)):
        # найдём размер матрицы для каждого пользователя
        sites_number = sites.shape[0]
        rows = sites_number // session_length if sites_number % session_length == 0 else sites_number // session_length + 1
        length = rows * session_length
        # создать numpy вектор для сессий
        sessions = np.empty(length, dtype=int)
        # остаток заполняем нулями
        zeros = length - sites_number
        if zeros > 0:
            sessions[-zeros:] = 0
        # кодировать названия сайтов
        for i, site in enumerate(sites):
            sessions[i] = frequency[site][0]
        # объединить сессии для каждого пользователя
        sessions = np.reshape(sessions, (rows, -1))
        sessions_dataframe = pd.DataFrame(sessions, columns=columns)
        sessions_dataframe['user_id'] = user_ids[index]
        features_list.append(sessions_dataframe)
    # получим features
    features = features_list[0].append(features_list[1:], ignore_index=True)
    
    return features 

In [ ]:
df = prepare_train_set(os.path.join(PATH_TO_DATA))

In [7]:
def prepare_train_set(path_to_csv_files, session_length=10):
    path_template = os.path.join(path_to_csv_files, os.path.normcase('user*.csv'))
    # создадим и отсортируем список файлов лексикографически
    csv_files = glob(path_template)
    csv_files.sort()
    # создадим список DataFrames из csv_files
    series = []
    for file in tqdm_notebook(csv_files):
        data = pd.read_csv(file)
        series.append(data['site'])
    # посчитаем частоту - 'frequency'
    frequency_dataframe = series[0].append(series[1:]).value_counts()
    frequency = {key: (index, frequency_dataframe[key]) for index, key
                 in enumerate(frequency_dataframe.index, start=1)}
    # создаём список user_ids
    user_ids = [int(re.search('([1-9][0-9]*).csv$', path).group(1)) for path in csv_files]
    # словарь уникальных сайтов
    columns = ['site' + str(i) for i in range(1, session_length + 1)]
    features_list = []
    for index, sites in tqdm_notebook(enumerate(series)):
        # найдём размер матрицы для каждого пользователя
        sites_number = sites.shape[0]
        rows = sites_number // session_length if sites_number % session_length == 0 else sites_number // session_length + 1
        length = rows * session_length
        # создать numpy вектор для сессий
        sessions = np.empty(length, dtype=int)
        # остаток заполняем нулями
        zeros = length - sites_number
        if zeros > 0:
            sessions[-zeros:] = 0
        # кодировать названия сайтов
        for i, site in enumerate(sites):
            sessions[i] = frequency[site][0]
        # объединить сессии для каждого пользователя
        sessions = np.reshape(sessions, (rows, -1))
        sessions_dataframe = pd.DataFrame(sessions, columns=columns)
        sessions_dataframe['user_id'] = user_ids[index]
        features_list.append(sessions_dataframe)
    # получим features
    features = features_list[0].append(features_list[1:], ignore_index=True)
    
    return features, frequency    

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [8]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

In [9]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,4,2,2,7,2,1,10,5,8,11,1
1,4,1,1,1,0,0,0,0,0,0,1
2,4,2,6,6,2,0,0,0,0,0,2
3,3,1,2,1,2,1,1,5,9,3,3
4,3,1,2,0,0,0,0,0,0,0,3


Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

In [10]:
site_freq_3users

{'google.com': (1, 9),
 'oracle.com': (2, 8),
 'meduza.io': (3, 3),
 'vk.com': (4, 3),
 'mail.google.com': (5, 2),
 'football.kulichki.ru': (6, 2),
 'geo.mozilla.org': (7, 1),
 'apis.google.com': (8, 1),
 'yandex.ru': (9, 1),
 'accounts.google.com': (10, 1),
 'plus.google.com': (11, 1)}

Примените полученную функцию к данным по 10 пользователям.

**<font color='red'> Вопрос 1. </font> Сколько уникальных сессий из 10 сайтов в выборке с 10 пользователями?**

In [11]:
train_data_10users, site_freq_10users = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 
                                                     session_length=10)

In [12]:
train_data_10users, site_freq_10users = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'))

In [13]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [14]:
# Число уникальных сессий из 10 сайтов
train_data_10users.shape[0]

14061

In [15]:
write_answer_to_file(train_data_10users.shape[0], 
                     'answers/answer1_1.txt')

**<font color='red'> Вопрос 2. </font> Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [16]:
# Число уникальных сайтов
len(site_freq_10users.keys())

4913

In [17]:
write_answer_to_file(len(site_freq_10users.keys()), 
                     'answers/answer1_2.txt')

Примените полученную функцию к данным по 150 пользователям.

**<font color='red'> Вопрос 3. </font> Сколько уникальных сессий из 10 сайтов в выборке с 150 пользователями?**

In [18]:
%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'))


Wall time: 2.36 s


In [19]:
# Число уникальных сессий
train_data_150users.shape[0]

137019

In [20]:
write_answer_to_file(train_data_150users.shape[0], 
                     'answers/answer1_3.txt')

**<font color='red'> Вопрос 4. </font> Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [21]:
len(site_freq_150users.keys())

27797

In [22]:
write_answer_to_file(len(site_freq_150users.keys()), 
                     'answers/answer1_4.txt')

**<font color='red'> Вопрос 5. </font> Какой из этих сайтов НЕ входит в топ-10 самых популярных сайтов среди посещенных 150 пользователями?**
- www.google.fr
- www.youtube.com
- safebrowsing-cache.google.com
- www.linkedin.com

топ-10 самых популярных сайтов среди посещенных 150 пользователями

In [23]:
top10_popular = ' '.join(list(zip(*sorted(site_freq_150users.items(),
                                         key=lambda kv: kv[1][1],
                                         reverse=True)[:10]))[0])

In [24]:
top10_popular

'www.google.fr www.google.com www.facebook.com apis.google.com s.youtube.com clients1.google.com mail.google.com plus.google.com safebrowsing-cache.google.com www.youtube.com'

In [25]:
write_answer_to_file(top10_popular, 
                     'answers/answer1_5.txt')

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [26]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. 

Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить. 

In [27]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [28]:
X_toy

array([[ 4,  2,  2,  7,  2,  1, 10,  5,  8, 11],
       [ 4,  1,  1,  1,  0,  0,  0,  0,  0,  0],
       [ 4,  2,  6,  6,  2,  0,  0,  0,  0,  0],
       [ 3,  1,  2,  1,  2,  1,  1,  5,  9,  3],
       [ 3,  1,  2,  0,  0,  0,  0,  0,  0,  0]])

In [29]:
X_sparse_toy = csr_matrix((np.ones(X_toy.size, dtype=int),
                          X_toy.reshape(-1),
                          np.arange(X_toy.shape[0] + 1) * X_toy.shape[1]))[:, 1:] 

**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [30]:
X_sparse_toy.todense()

matrix([[1, 3, 0, 1, 1, 0, 1, 1, 0, 1, 1],
        [3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0],
        [4, 2, 2, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [31]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [32]:
X_sparse_10users = csr_matrix((np.ones(X_10users.size, dtype=int),
                              X_10users.reshape(-1),
                              np.arange(X_10users.shape[0] + 1) * X_10users.shape[1]))[:, 1:]
X_sparse_150users = csr_matrix((np.ones(X_150users.size, dtype=int),
                              X_150users.reshape(-1),
                              np.arange(X_150users.shape[0] + 1) * X_150users.shape[1]))[:, 1:]

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [33]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [91]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [92]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

## Пути улучшения
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- помимо явного создания разреженного формата можно еще составить выборки с помощью `CountVectorizer`, `TfidfVectorizer` и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle Inclass (желающие могут начать уже сейчас).

На следующей неделе мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 